In [1]:
import $file.hw7stdlib
import hw7stdlib._


import $file.$        

import hw7stdlib._


In [2]:
sealed trait Color
case object Red extends Color
case object Green extends Color
case object Blue extends Color
case object Yellow extends Color
case object Cyan extends Color
case object Magenta extends Color
case object Clear extends Color
case object White extends Color
case object Black extends Color
case class RGB(r : Double, g : Double, b : Double) extends Color

sealed trait Primitive
case class Box(x1 : Double, y1 : Double, z1 : Double, x2 : Double, y2 : Double, z2 : Double) extends Primitive
case class Sphere(x : Double, y : Double, z : Double, r : Double) extends Primitive
case class Cone(x1 : Double, y1 : Double, z1 : Double, r1 : Double, x2 : Double, y2 : Double, z2 : Double, r2 : Double) extends Primitive
case class Torus(major : Double, minor : Double) extends Primitive
case class Cylinder(x1 : Double, y1 : Double, z1 : Double, x2 : Double, y2 : Double, z2 : Double, r2 : Double) extends Primitive

sealed trait Transformation
case class Translate(x : Double, y : Double, z : Double) extends Transformation
case class Rotate(x : Double, y : Double, z : Double) extends Transformation
case class Scale(x : Double, y : Double, z : Double) extends Transformation

sealed trait Expr
case class Const(x : Integer) extends Expr
case class Bin(x : Bool) extends Expr
case class Ident(x : String) extends Expr
case class Plus(x : Expr, y : Expr) extends Expr
case class Minus(x : Expr, y : Expr) extends Expr
case class Mult(x : Expr, y : Expr) extends Expr
case class Pow(x : Expr, y : Expr) extends Expr
case class Neg(x : Expr) extends Expr
case class Eq(x : Expr, y : Expr) extends Expr
case class And(x : Expr, y : Expr) extends Expr
case class Or(x : Expr, y : Expr) extends Expr
case class IfThenElse(p : Expr, t : Expr, f : Expr) extends Expr
case class Let(id : String, x : Expr, y : Expr) extends Expr
case class FunDef(id : String, x : Expr) extends Expr
case class FunCall(x : Expr, y : Expr) extends Expr
case class LetRec(id1 : String, id2 : String, x : Expr, y : Expr) extends Expr

case class Sequence(in : List[Expr]) extends Expr
case class While(in : Expr, body : Expr) extends Expr

case class Object(in : Primitive, inc : Color) extends Expr
case class Union(p1 : Expr, p2 : Expr) extends Expr
case class Intersection(p1 : Expr, p2 : Expr) extends Expr
case class Difference(p1 : Expr, p2 : Expr) extends Expr
case class Transform(in : Expr, trans : Transformation) extends Expr

defined trait Color
defined object Red
defined object Green
defined object Blue
defined object Yellow
defined object Cyan
defined object Magenta
defined object Clear
defined object White
defined object Black
defined class RGB
defined trait Primitive
defined class Box
defined class Sphere
defined class Cone
defined class Torus
defined class Cylinder
defined trait Transformation
defined class Translate
defined class Rotate
defined class Scale
defined trait Expr
defined class Const
defined class Bin
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Pow
defined class Neg
defined class Eq
defined class And
defined class Or
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec
defined class Sequence
defined class While
defined class Object
defined class Union
defined class Intersection
defined class Difference
defined class Transform

In [3]:
sealed trait Environment
case object EmptyEnv extends Environment 
case class Extend(k : String, v : Value, env : Environment) extends Environment
case class ExtendRec(f: String, x: String, e: Expr, sigma: Environment ) extends Environment

sealed trait Value
case class NumVal(x : Integer) extends Value 
case class BinVal(x : Bool) extends Value
case class CSGVal(x : String) extends Value
case object Error extends Value
case class Closure(x : String, y : Expr, z : Environment) extends Value

object Writer {
    private val _init_str : String = "  #include \"colors.inc\"\n  camera {\nlocation <0, 1, -10>\nlook_at 0\nangle 36\n}\nlight_source { <500, 500, -1000> White }\n"
    private var _str : String = _init_str
    
    def Write(x : Value, name : String) : Unit = x match {
            case CSGVal(x) => {
                this.AddString(x)
                
                import java.nio.file
                val wr = java.nio.file.Files.newBufferedWriter(java.nio.file.Paths.get(name))
                try wr.write(_str) finally wr.close()
            }
            case _         => x
        }
    def AddString(x : String) : Unit = _str += x
    def ClearString() : Unit = _str = _init_str
    def GetString() : String = _str
}

defined trait Environment
defined object EmptyEnv
defined class Extend
defined class ExtendRec
defined trait Value
defined class NumVal
defined class BinVal
defined class CSGVal
defined object Error
defined class Closure
defined object Writer

In [4]:
def lookupEnv(sigma: Environment, x: String): Maybe[Value] = sigma match {
    case EmptyEnv => Nothing
    case Extend(y, v, pi) => string_eq(y,x) match {
        case True => Just(v)
        case False => lookupEnv(pi, x)
    } 
    case ExtendRec(f, y, e, pi) => string_eq(x,f) match {
        case True => Just(Closure(y, e, sigma))
        case False => lookupEnv(pi, x)
    } 
}   

defined function lookupEnv

In [31]:
def eval(env : Environment, expr : Expr) : Value = expr match {
    case Const(n)  => NumVal(n)
    case Bin(p)    => BinVal(p)
    case Ident(id) => lookupEnv(env, id) match {
        case Just(v) => v
        case Nothing => Error
    }
    case Plus(e1, e2) => eval_bin_arith(plus, env, e1, e2)
    case Minus(e1, e2) => eval_bin_arith(minus, env, e1, e2)
    case Mult(e1, e2) => eval_bin_arith(mult, env, e1, e2)
    case Pow(e1, e2)  => (eval(env, e1), eval(env,e2)) match {
        case (NumVal(x), NumVal(Positive(n))) => NumVal(pow(x, n))
        case _ => Error
    }
    case Neg(e) => eval(env, e) match {
        case NumVal(x) => NumVal(negate(x))
        case BinVal(p) => BinVal(not(p))
        case Error     => Error
    }
    case Eq(e1, e2) => (eval(env, e1), eval(env, e2)) match {
        case (NumVal(x), NumVal(y)) => BinVal(int_eq(x,y))
        case (BinVal(p), BinVal(q)) => BinVal(bool_eq(p,q))
        case _                      => Error
    }
    case And(e1, e2) => eval_bin_bool(and, env, e1, e2)
    case Or(e1, e2) => eval_bin_bool(or, env, e1, e2)
    case IfThenElse(p, e_t, e_f) => eval(env, p) match{
        case BinVal(True)  => eval(env, e_t)
        case BinVal(False) => eval(env, e_f)
        case _             => Error
    }
    case Let(id, df, body) => eval(env, df) match{
        case Error => Error
        case x     => {
            val new_env = Extend(id, x, env)
            eval(new_env, body) 
        }
    }
    // FunDef, FunCall, and LetRec cases go here
    case FunDef(arg, body)  => Closure(arg, body, env)
    case FunCall(func, arg) => (eval(env, func), eval(env, arg)) match {
       case (_, Error)                       => Error
       case (Closure(arg, body, env), f_arg) => eval(Extend(arg, f_arg, env), body)
       //The function called is not a closure
       case (_, _)                           => Error
    }
    //huh?
    case LetRec(f, param, body, prgm) => eval(ExtendRec(f, param, body, env), prgm)
    case Sequence(x) => x match {
        case Empty          => Error
        case Cons(x, Empty) => eval(env, x)
        case Cons(x, next)  => x match {
            case Let(id, df, body)            => eval(Extend(id, eval(env, df), env), Sequence(Cons(body, next)))
            case LetRec(f, param, body, prgm) => eval(ExtendRec(f, param, body, env), Sequence(next))
            case _                            => {
                eval(env, x) match {
                    case CSGVal(x) => Writer.AddString(x)
                    case x => x
                }
                eval(env, Sequence(next))
            }
        }
    }
    case While(x, body) => eval(env, x) match {
        case BinVal(y) => y match {
            case True  => eval(env, Sequence(Cons(body, 
                                               Cons(While(x, body), Empty))))
                          //eval(env, LetRec("while", "param", body, )
            case False => BinVal(False)
        }
        case _         => Error
    }
    case Object(x, color) => x match {
        case Box(x1, y1, z1, x2, y2, z2) 
            => CSGVal("box {\n <" + x1.toString() + ", " + y1.toString() + ", " + z1.toString() + ">,\n <"
                      + x2.toString() + ", " + y2.toString() + ", " + z2.toString() + ">\n pigment { "
                      + color.toString() + " }\n}\n")
        case Sphere(x, y, z, r) 
            => CSGVal("sphere {\n <" + x.toString() + ", " + y.toString() + ", " 
                      + z.toString() + ">, " + r.toString() + "\n pigment { "
                      + color.toString() + " }\n}\n")
        case Cone(x1, y1, z1, r1, x2, y2, z2, r2) 
            => CSGVal("cone {\n <" + x1.toString() + ", " + y1.toString() + ", " + z1.toString() + ">, "
                      + r1.toString() + "\n <" + x2.toString() + ", " + y2.toString() + ", " + z2.toString() + ">, "
                      + r2.toString() + "\n pigment { " + color.toString() + " }\n}\n")
        case Torus(x, y) 
            => CSGVal("torus {\n " + x.toString() + ", " + y.toString() + "\n pigment { "
                      + color.toString() + " }\n}\n")
        case Cylinder(x1, y1, z1, x2, y2, z2, r) 
            => CSGVal("cylinder {\n <" + x1.toString() + ", " + y1.toString() + ", " + z1.toString() + ">,\n <"
                      + x2.toString() + ", " + y2.toString() + ", " + z2.toString() + ">,\n " 
                      + r.toString() + "\n pigment { " + color.toString() + " }\n}\n")
    }
    case Union(e1, e2) => (eval(env, e1), eval(env, e2)) match {
        case (CSGVal(x), CSGVal(y)) => CSGVal("merge {\n " + x + "\n\n " + y + "\n}\n")
        case _ => Error
    }
    case Difference(e1, e2) => (eval(env, e1), eval(env, e2)) match {
        case (CSGVal(x), CSGVal(y)) => CSGVal("difference {\n " + x + "\n\n " + y + "\n}\n")
        case _ => Error
    }
    case Intersection(e1, e2) => (eval(env, e1), eval(env, e2)) match {
        case (CSGVal(x), CSGVal(y)) => CSGVal("intersection {\n" + x + "\n\n " + y + "\n}\n")
        case _ => Error
    }
    case Transform(e, trans) => (eval(env, e), trans) match {
        case (CSGVal(a), Translate(x, y, z)) => CSGVal(a.dropRight(2) + " translate <" + x.toString() +
                                                       ", " + y.toString() + ", " + z.toString() + "> \n}\n")
        case (CSGVal(a), Scale(x, y, z))     => CSGVal(a.dropRight(2) + " scale <" + x.toString() +
                                                       ", " + y.toString() + ", " + z.toString() + "> \n}\n")
        case (CSGVal(a), Rotate(x, y, z))    => CSGVal(a.dropRight(2) + " rotate <" + x.toString() +
                                                       ", " + y.toString() + ", " + z.toString() + "> \n}\n")
        case _ => Error
    }
}
def eval_bin_arith( op : (Integer, Integer) => Integer
                  , env : Environment
                  , e1 : Expr
                  , e2 : Expr) : Value 
    = (eval(env, e1), eval(env, e2)) match{
        case (NumVal(x), NumVal(y)) => NumVal(op(x,y))
        case _ => Error
    }

def eval_bin_bool( op : (Bool, Bool) => Bool
                 , env : Environment
                 , e1 : Expr
                 , e2 : Expr) : Value 
    = (eval(env, e1), eval(env, e2)) match{
        case (BinVal(x), BinVal(y)) => BinVal(op(x,y))
        case _ => Error
    }

defined function eval
defined function eval_bin_arith
defined function eval_bin_bool

In [33]:
/*val a : Expr = Sequence(Cons(Object(Torus(1.0, 2.0)), 
                                          Cons(Object(Sphere(1,1,1,2.5)), 
                                               Cons(Object(Torus(5.0, -5.0)), Empty))))
val b : Expr = Transform(Intersection(Transform(Object(Sphere(0,0,0,1.5)),Scale(1,2,3)),
                                      Object(Box(-1,-1,-1,1,1,1))),Rotate(1,2,3))
val c : Expr = Sequence(Cons(Transform(Object(Torus(1.0, 2.0)),Rotate(0,5,0)), 
                                          Cons(Object(Sphere(1,1,1,2.5)), 
                                               Cons(Object(Torus(5.0, -5.0)), Empty)))) */
val d : Expr = Object(Sphere(1,1,1,2.5), Red)
val t : Expr = Sequence(Cons(Let("x", d, Union(d, d)),
                                Cons(Object(Torus(2,5), Blue), Empty)))

val e : Expr = Sequence(Cons(Let("x", d, Ident("x")),
                            Cons(Transform(Difference(Ident("x"), Object(Torus(1, 1), Green)), Rotate(0.0, 0.0, 90.0)),
                                Cons(Object(Box(-5,-5,-5,-3,-3,-3), Blue), Empty))))
val f : Expr = Sequence(Cons(d, Empty))
val g : Expr = Let("x", Const(Positive(Zero)),
                   While(Neg(Eq(Ident("x"), Const(Positive(five)))),
                        Let("x", Plus(Ident("x"), Const(Positive(one))), d)))
val union_demo : Expr = Union(Object(Sphere(-2,0,3,2.5), Red), Object(Sphere(2,0,3,2.5), Blue))
val intersection_demo : Expr = Intersection(Object(Sphere(-2,0,3,2.5), Red), Object(Sphere(2,0,3,2.5), Blue))
val difference_demo : Expr = Difference(Object(Sphere(-2,0,3,2.5), Red), Object(Sphere(2,0,3,2.5), Blue))




Writer.ClearString()
//Writer.Write(eval(EmptyEnv, e), "DSL.inc")
//Writer.Write(eval(EmptyEnv, difference_demo), "difference_demo.inc")
Writer.Write(eval(EmptyEnv, g), "test.inc")
//eval(EmptyEnv, Union(Object(Torus(2.0, 1.0)), Object(Torus(1.0, -2.0))))
Writer.GetString()

d: Expr = Object(Sphere(1.0, 1.0, 1.0, 2.5), Red)
t: Expr = Sequence(
  Cons(
    Let(
      "x",
      Object(Sphere(1.0, 1.0, 1.0, 2.5), Red),
      Union(
        Object(Sphere(1.0, 1.0, 1.0, 2.5), Red),
        Object(Sphere(1.0, 1.0, 1.0, 2.5), Red)
      )
    ),
    Cons(Object(Torus(2.0, 5.0), Blue), Empty)
  )
)
e: Expr = Sequence(
  Cons(
    Let("x", Object(Sphere(1.0, 1.0, 1.0, 2.5), Red), Ident("x")),
    Cons(
      Transform(
        Difference(Ident("x"), Object(Torus(1.0, 1.0), Green)),
        Rotate(0.0, 0.0, 90.0)
      ),
      Cons(Object(Box(-5.0, -5.0, -5.0, -3.0, -3.0, -3.0), Blue), Empty)
    )
  )
)
f: Expr = Sequence(Cons(Object(Sphere(1.0, 1.0, 1.0, 2.5), Red), Empty))
g: Expr = Let(
  "x",
  Const(Positive(Zero)),
  While(
    Neg(Eq(Ident("x"), Const(Positive(Succ(Succ(Succ(Succ(Succ(Zero))))))))),
    Let(
      "x",
      Plus(Ident("x"), Const(Positive(Succ(Zero)))),
      Object(Sphere(1.0, 1.0, 1.0, 2.5), Red)
    )
  )
)
union_demo: Expr = Union(
  